In [1]:
import re
import random
import math
import json
import jieba

import numpy as np
from tqdm import tqdm
from transformers import BertTokenizer

2022-10-02 01:00:49.371865: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2022-10-02 01:00:49.371901: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
model_checkpoint =  '/root/autodl-tmp/pretrained_models/bert_large_UER'

tokenizer = BertTokenizer.from_pretrained(model_checkpoint)

In [3]:
seed_value = 1227927
random.seed(seed_value) # Python
np.random.seed(seed_value) # cpu vars
# torch.manual_seed(seed_value) # cpu  vars

In [4]:
with open('./data/发明专利数据.json', 'r') as f:
    train_data_patent = f.readlines()
#     json.load(f)

In [5]:
with open('/root/autodl-tmp/CCF-小样本/data/train.json', 'r')as f:
    train_data = f.readlines()
    train_data = [eval(i.strip())for i in train_data]
    
with open('/root/autodl-tmp/CCF-小样本/data/testA.json', 'r')as f:
    test_data = f.readlines()
    test_data = [eval(i.strip())for i in test_data]

In [6]:
# dd = eval(str(train_data_patent))
# dd = str()
train_data_patent = str(train_data_patent)


data = re.compile('{.*?}').findall(train_data_patent)

In [7]:
clean_data = []
train_data_ = []
for i in tqdm(data):
    tmp = dict()
    pat_applicant = re.compile('"pat_applicant":"(.*?)",').findall(i)
    pat_name = re.compile('"pat_name":"(.*?)",').findall(i)
    pat_summary = re.compile('"pat_summary":"(.*?)",').findall(i)
    
    if pat_applicant:
        tmp['assignee'] = pat_applicant[0]
    else:
        tmp['assignee'] = ''
        
        
    if pat_name:
        tmp['title'] = pat_name[0]
        train_data_.append(pat_name[0])
    else:
        tmp['title'] = ''

    if pat_summary:
        tmp['abstract'] = pat_summary[0]
        train_data_.append(pat_summary[0])
    else:
        tmp['abstract'] = ''
    
    clean_data.append(tmp)

100%|██████████| 281460/281460 [00:03<00:00, 77500.37it/s]


In [8]:
clean_data[0], train_data[0]

({'assignee': '重庆燃气集团股份有限公司',
  'title': '一种燃气管封堵装置及封堵方法',
  'abstract': '一种燃气管封堵装置，涉及燃气工程领域，其包括动力装置、弹性垫和压板；动力装置包括缸体、缸盖、活塞及活塞杆；缸体内设有腔室，缸体具备第一端面及第二端面，活塞杆包括第一端及第二端；活塞与活塞杆固定连接，并且活塞杆贯穿缸体；弹性垫设置在压板及第一端面之间。该装置结构简单使用方便，能够快速的与燃气管对接，并且通过动力装置能够与燃气管内壁紧密的贴合，封堵效果良好，能够有效防止泄漏。基于上述的燃气管封堵装置，本发明还提供了一种封堵方法。'},
 {'id': '538f267d2e6fba48b1286fb7f1499fe7',
  'title': '一种信号的发送方法及基站、用户设备',
  'assignee': '华为技术有限公司',
  'abstract': '一种信号的发送方法及基站、用户设备。在一个子帧中为多个用户设备配置的参考信号的符号和数据的符号在子帧中的时域位置关系满足前提一和前提二；前提一为，将每个用户设备的参考信号所需的资源包括在多个参考信号的符号中，前提二为以下条件中的至少一个：将每个用户设备的多个参考信号设置在每个用户设备的数据的符号之前的参考信号的符号中，和/或每个用户设备的数据的符号之后的参考信号的符号中，从而有效地节省了发送参考信号的开销，满足了资源设计的需求；且部分或全部用户设备可在多个参考信号的符号中包含其参考信号，使该用户设备的解调性能得到进一步改善。',
  'label_id': 0})

In [9]:
patent_titles = set()


for i in train_data:
    patent_titles.add(i['title'])
    
for i in test_data:
    patent_titles.add(i['title'])
    
for i in clean_data:
    patent_titles.add(i['title'])
    
    

In [11]:
pretrain_data = []

cut_text_len = 506

for i in tqdm(train_data + test_data + clean_data):
    
    if not i['title'] or not i['abstract']: continue
    
    title = i['title']
    abstract = i['abstract']
    assignee = i['assignee']
    label = '1'
    if random.random() > 10: ####################
        sample_title = ''
        while sample_title == title or sample_title: 
            sample_title = random.sample(patent_titles, 1)[0]
        title = sample_title
        label = '0'
        

        
    title_len = len(title)
    abstract_len = len(abstract)
    assignee_len = len(assignee)
    cut_text_len_ = cut_text_len - title_len - assignee_len
    
    
    cut_count = math.ceil(abstract_len / cut_text_len_)
    
    for j in range(cut_count):
        start = j * cut_text_len_
        end = (j+1) * cut_text_len_
        
        sub_abstract = abstract[start: end]
        
        
        if len(sub_abstract) > 10:
            tmp = title + '---' + assignee + '---' + sub_abstract + '---' + label
            pretrain_data.append(tmp)

# for i in clean_data:
#     title = i['pat_name']
#     abstract = i['abstract']
#     label = '1'    
    


100%|██████████| 303257/303257 [00:01<00:00, 221726.04it/s]


In [12]:
random.shuffle(pretrain_data)

In [13]:
len(pretrain_data)

303486

In [14]:
with open('./abstracts_new.txt', 'w') as f:
    f.write('\n'.join(pretrain_data))
    
with open('./abstracts_new_test.txt', 'w') as f:
    f.write('\n'.join(pretrain_data[:5000]))

In [ ]:
for idx, line in tqdm(enumerate(pretrain_data[:])):
        words_ids = []
        line = line.strip().split('---')
        title = line[0]
        assignee = line[1]
        abstract = line[2]
        label = line[3]

        words_ids = []
        tokens_title = tokenizer.tokenize(title)
        tokens_assignee= tokenizer.tokenize(assignee)
        tokens_abstract = tokenizer.tokenize(abstract)

        tokens = ["[CLS]"] + tokens_title + ["[SEP]"] + tokens_assignee + ["[SEP]"] + tokens_abstract + ["[SEP]"]
        
        if len(tokens) >= 512:
            print(idx, line)
            break

76395it [03:16, 343.17it/s]

In [31]:
len(abstract)

505

In [7]:
with open('./data/patent_data.json', 'w') as f:
    f.write('\n'.join([str(i) for i in clean_data]))

In [8]:
with open('./abstracts.txt', 'r') as f:
    train_data = f.readlines()

In [9]:
train_data = train_data + train_data_

In [10]:
new_data = []
cut_text_len = 510

for content in tqdm(train_data):
    content_len = len(content)
    
    new_content = []
    cun_count = math.ceil(content_len / cut_text_len)
    
    for i in range(cun_count):
        start = i * cut_text_len
        end = (i+1) * cut_text_len
        sub_content = content[start: end]
        sub_content_len = len(sub_content)
        mid_content_len = int(sub_content_len / 2)
        new_data.append(sub_content)
        
#         if sub_content_len > 256 and random.random() > 0.9:
#             cut_num = random.choice(range(mid_content_len, mid_content_len + random.choice(range(5, 20))))
#             new_data.append(sub_content[: cut_num])
#             new_data.append(sub_content[cut_num: ])

#         else:
#             new_data.append(sub_content)

    

100%|██████████| 584490/584490 [00:01<00:00, 394075.57it/s]


In [11]:
!pwd

/root/autodl-tmp/CCF-小样本/Nezha_pytorch/pretrain


In [12]:
new_data = [i.strip() for i in new_data if i.strip()]
len(new_data)

584814

In [13]:
with open('./abstracts_new.txt', 'w') as f:
    f.write('\n'.join(new_data))

In [14]:
with open('./abstracts_new_test.txt', 'w') as f:
    f.write('\n'.join(new_data[:10000]))

In [15]:
with open('./abstracts_new_test.txt', 'r') as f:
    abstracts_new_test = f.readlines()

In [16]:
abstracts_new_test[:2]

['一种信号的发送方法及基站、用户设备。在一个子帧中为多个用户设备配置的参考信号的符号和数据的符号在子帧中的时域位置关系满足前提一和前提二；前提一为，将每个用户设备的参考信号所需的资源包括在多个参考信号的符号中，前提二为以下条件中的至少一个：将每个用户设备的多个参考信号设置在每个用户设备的数据的符号之前的参考信号的符号中，和/或每个用户设备的数据的符号之后的参考信号的符号中，从而有效地节省了发送参考信号的开销，满足了资源设计的需求；且部分或全部用户设备可在多个参考信号的符号中包含其参考信号，使该用户设备的解调性能得到进一步改善。\n',
 '本发明公开了一种5G通讯电缆故障监控系统，包括信号采样模块、补偿反馈模块，所述信号采样模块对5G通讯电缆信号采样，信号采样模块连接补偿反馈模块，补偿反馈模块运用三极管Q1、电容C3和电感L2、电容C2组成高频补偿电路展宽信号的通频带，为了进一步保证滤除扰动信号的准确性，避免异常高电平信号击穿电感L3，运用三极管Q4检测运放器AR2输出端信号，将异常高电平信号经电阻R14分压，最后运用运放器AR3同相放大信号，三极管Q5进一步三极管运放器AR3输出信号、三极管Q3发射极信号电位差，运用三极管Q2反馈信号至运放器AR2输出端，对运放器AR3输出信号峰值进一步校准，5G通讯电缆故障监控系统终端能够及时对5G通讯电缆故障及时响应。\n']

In [10]:
new_data_[:5]

['一种信号的发送方法及基站、用户设备。在一个子帧中为多个用户设备配置的参考信号的符号和数据的符号在子帧中的时域位置关系满足前提一和前提二；前提一为，将每个用户设备的参考信号所需的资源包括在多个参考信号的符号中，前提二为以下条件中的至少一个：将每个用户设备的多个参考信号设\n',
 '置在每个用户设备的数据的符号之前的参考信号的符号中，和/或每个用户设备的数据的符号之后的参考信号的符号中，从而有效地节省了发送参考信号的开销，满足了资源设计的需求；且部分或全部用户设备可在多个参考信号的符号中包含其参考信号，使该用户设备的解调性能得到进一步改善。\n',
 '\n',
 '本发明公开了一种5G通讯电缆故障监控系统，包括信号采样模块、补偿反馈模块，所述信号采样模块对5G通讯电缆信号采样，信号采样模块连接补偿反馈模块，补偿反馈模块运用三极管Q1、电容C3和电感L2、电容C2组成高频补偿电路展宽信号的通频带，为了进一步保证滤除扰动信号的准确性，避免异常高电平信号击穿电感L3，运用三极管Q4检测运放器AR2输出端信号，将异常高电平\n',
 '信号经电阻R14分压，最后运用运放器AR3同相放大信号，三极管Q5进一步三极管运放器AR3输出信号、三极管Q3发射极信号电位差，运用三极管Q2反馈信号至运放器AR2输出端，对运放器AR3输出信号峰值进一步校准，5G通讯电缆故障监控系统终端能够及时对5G通讯电缆故障及时响应。\n']